# Pipeline End-to-End: YOLO → ResNet (Translation + Rotation) → 6D Pose (Minimal Version)
Questa versione minimale esegue la pipeline end-to-end per la stima della posa 6D su LineMOD, ispirata al notebook test6_extension_endtoend_pipeline.ipynb.

**Nota:** Questa pipeline minimale esegue solo un batch di test e mostra la traduzione e la rotazione predette. Per valutazioni complete e metriche, vedere il notebook originale test6_extension_endtoend_pipeline.ipynb.

## 1. Import essenziali e setup minimal

In [ ]:
# Import essenziali e setup minimal
import os
import sys
import pandas as pd
from pathlib import Path
import torch

sys.path.insert(0, str(Path.cwd().parent))
from config import Config
from models.pose_estimator_endtoend import PoseEstimator
from models.yolo_detector import YOLODetector
from dataset.linemod_pose import create_pose_dataloaders
from utils.validation import run_deep_pose_pipeline, load_validation_results, calc_add_accuracy_per_class
from utils.visualization import plot_add_per_class, plot_pinhole_error_per_class


## 2. Carica modelli pre-addestrati (YOLO e PoseEstimator End-to-End)

In [ ]:
# Carica modelli pre-addestrati (YOLO e PoseEstimator End-to-End)
yolo_weights = Config.CHECKPOINT_DIR / 'pretrained' / 'yolo11n.pt'
yolo_detector = YOLODetector(model_name=str(yolo_weights), num_classes=Config.NUM_CLASSES) if yolo_weights.exists() else None
pose_ckpt = Config.CHECKPOINT_DIR / 'pose' / 'test_endtoend_pose_1' / 'weights' / 'best.pt'
model_endtoend = PoseEstimator(pretrained=True).to(Config.DEVICE)
if pose_ckpt.exists():
    model_endtoend.load_state_dict(torch.load(pose_ckpt, map_location=Config.DEVICE))
model_endtoend.eval()

## 3. Carica un batch di test e applica la pipeline end-to-end minimale

In [ ]:
# Carica un batch di test e applica la pipeline end-to-end minimale
train_loader, val_loader, test_loader = create_pose_dataloaders(
    dataset_root=Config.LINEMOD_ROOT,
    batch_size=1,
    crop_margin=Config.POSE_CROP_MARGIN,
    output_size=Config.POSE_IMAGE_SIZE,
    num_workers=0
)
batch = next(iter(test_loader))
image = batch['rgb_crop'].to(Config.DEVICE)
if yolo_detector:
    det = yolo_detector.detect(image)
else:
    det = None  # fallback: nessuna detection
if det is not None:
    # Qui si dovrebbe croppare l'immagine secondo bbox, ma per minimalità usiamo l'immagine già croppata
    pred_quat, pred_trans = model_endtoend(image)
else:
    pred_quat, pred_trans = model_endtoend(image)
print('Predicted translation:', pred_trans.cpu().detach().numpy())
print('Predicted rotation (quaternion):', pred_quat.cpu().detach().numpy())

## 4. Esegui la pipeline end-to-end su tutto il test set e salva le metriche (ADD full pose)

In [ ]:
# Esegui la pipeline end-to-end su tutto il test set e salva le metriche (ADD full pose)
run_deep_pose_pipeline(model_endtoend, test_loader, name='test_endtoend_pose_1')

## 5. Carica e mostra tabella delle metriche per classe (ADD full pose)

In [ ]:
# Carica e mostra tabella delle metriche per classe (ADD full pose)
val_csv_path = os.path.join(Config.CHECKPOINT_DIR, 'pose', 'test_endtoend_pose_1', 'validation_result.csv')
results_full_pose, _ = load_validation_results(val_csv_path)

data, global_add, global_acc = calc_add_accuracy_per_class(results_full_pose, Config.LINEMOD_OBJECTS)

df = pd.DataFrame(data)
display(df)
print(f"\nMedia globale ADD (full pose): {global_add:.2f}")
print(f"Accuracy globale (full pose) (%): {global_acc:.1f}")

## 6. Grafico a barre delle medie ADD per classe
Mostra le medie della metrica ADD per ciascuna classe oggetto, calcolate sulla rotazione-traslazione predetta.

In [ ]:
# Grafico a barre delle medie ADD per classe (rot-only)
results_rot_only = globals().get('results_rot_only', None)
if results_rot_only is None:
    print("⚠️  Devi prima calcolare la metrica ADD rot-only su tutto il test set e salvare i risultati in 'results_rot_only'.")
else:
    plot_add_per_class(results_rot_only, Config.LINEMOD_OBJECTS)

# Grafico a barre dell'errore medio pinhole per classe
results_pinhole = globals().get('results_pinhole', None)
if results_pinhole is None:
    print("⚠️  Devi prima calcolare le statistiche di traslazione pinhole e salvarle in 'results_pinhole'.")
else:
    plot_pinhole_error_per_class(results_pinhole, Config.LINEMOD_OBJECTS)